<a href="https://colab.research.google.com/github/Kang-Beom-Seo/Korean-NLP-Practice/blob/main/%EC%BD%94%EC%82%AC%EC%9D%B8_%EC%9C%A0%EC%82%AC%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B)) #노름은 L2노름(유클리드 노름)

In [ ]:
doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])

In [ ]:
print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
data = pd.read_csv('/content/drive/My Drive/TensorFlow_tutorial/문서 유사도/movies_metadata.csv', low_memory=False)
data.head(2)

In [ ]:
data = data.head(20000) #20000개의 샘플만 이용

In [ ]:
data['overview'].isnull().sum() #overview 열에 null값이 총 135개 존재

In [ ]:
# overview에서 Null 값을 가진 경우에는 Null 값을 제거
data['overview'] = data['overview'].fillna('')

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape) # 총 47487개의 단어가 존재한다.

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim.shape

In [ ]:
indices = pd.Series(data.index, index=data['title']).drop_duplicates() #영화의 title을 입력시 인덱스가 리턴됨.
print(indices.head())

In [ ]:
idx = indices['Father of the Bride Part II']
print(idx)

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx])) #enumerate는 (index, value)의 형태로 열거된다.

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) #reverse=True이므로 내림차순

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11] #1부터 시작하는 이유는 아마 가장 유사한 영화는 그 영화 그자체이므로 제외하는 것이다.

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['title'].iloc[movie_indices]

In [ ]:
get_recommendations('The Dark Knight Rises')